# **IP3_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
Markdown(response1.choices[0].message.content)

Parameters:
- Set I: Set of available items to pack.
- Set B: Set of available boxes.
- W: The weight of item i Є I.
- C: The capacity of available boxes.
- M: The maximum number of available boxes.

Decision Variables:
- X_{ij}: This binary variable equals 1 if item i is placed in box j, and 0 otherwise.

Objective Function:
- The goal is to minimize the number of boxes used, which is Sum (for j Є B) (Sum (for i Є I) X_{ij}).

Constraints:
1. Item weight constraint: For each box j, the total weight of the items it contains must not exceed its capacity. 
   That is, Sum (for i Є I) (W_i * X_{ij}) ≤ C, for each j Є B.
   
2. Item placement constraint: Each item i can be placed in one box only, meaning Sum (for j Є B) X_{ij} = 1, for all i Є I.

3. Box constraint: The total number of boxes used cannot exceed the maximum limit. 
   That is, Sum (for j Є B) (Sum (for i Є I) X_{ij}) ≤ M.
   
4. Binary constraint: X_{ij} Є {0, 1}, for all i Є I, j Є B.

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

Based on your given optimization problem, here's a simple implementation using Pyomo in Python:

```python
# Import Pyomo
from pyomo.environ import *

# Data: this is where we fill in the actual data.
I = [1, 2, 3]       # Available items
B = [1, 2, 3, 4, 5]  # Available boxes
W = {1:4, 2:2, 3:7}  # Weight of each item
C = 10               # The capacity of each box
M = 3                # Maximum number of boxes

# Create a concrete model
model = ConcreteModel()

# Decision variables
model.X = Var(((i,j) for i in I for j in B), within=Binary)

# Objective function
model.obj = Objective(expr=sum(model.X[i,j] for i in I for j in B), sense=minimize)

# Constraints
model.item_placement = ConstraintList()
for i in I:
    model.item_placement.add(sum(model.X[i,j] for j in B) == 1)

model.box_capacity = ConstraintList()
for j in B:
    model.box_capacity.add(sum(W[i]*model.X[i,j] for i in I) <= C)

model.box_limit = Constraint(expr=sum(model.X[i,j] for i in I for j in B) <= M )

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of decision variables
for i in I:
    for j in B:
        print(f'X_{i}{j} = {value(model.X[i,j])}')
```

In this code, we import the `pyomo.environ` module and define a simple model that represents your packing problem. 

We start by defining the available items and boxes, and the weight for each item. `W` is a dictionary where the keys are the items and the values are their weights. `C` is the capacity of the boxes. `M` is the maximum number of boxes we can use. The sets `I` and `B` represent available items and boxes respectively.

For simplicity, I've assumed here that the maximum capacity of each box and the maximum number of boxes are constants. If these values differ per box, you'll need to make alterations to the model accordingly.

The objective function minimizes the number of boxes used, and the constraints enforce that each item is in exactly one box, the sum of the weights in each box doesn't exceed its capacity, and the total number of boxes used doesn't exceed `M`.

Finally, we solve the model using the GLPK solver and print the decision variables' optimal values. 

Note: GLPK is one of the solver supported by Pyomo. If this solver is not available in your machine, you might need to install it or you can use a different solver that's supported by your platform.

### **Run the code Generated by GPT4**

In [ ]:
# Import Pyomo
from pyomo.environ import *

# Data: this is where we fill in the actual data.
I = [1, 2, 3]       # Available items
B = [1, 2, 3, 4, 5]  # Available boxes
W = {1:4, 2:2, 3:7}  # Weight of each item
C = 10               # The capacity of each box
M = 3                # Maximum number of boxes

# Create a concrete model
model = ConcreteModel()

# Decision variables
model.X = Var(((i,j) for i in I for j in B), within=Binary)

# Objective function
model.obj = Objective(expr=sum(model.X[i,j] for i in I for j in B), sense=minimize)

# Constraints
model.item_placement = ConstraintList()
for i in I:
    model.item_placement.add(sum(model.X[i,j] for j in B) == 1)

model.box_capacity = ConstraintList()
for j in B:
    model.box_capacity.add(sum(W[i]*model.X[i,j] for i in I) <= C)

model.box_limit = Constraint(expr=sum(model.X[i,j] for i in I for j in B) <= M )

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of decision variables
for i in I:
    for j in B:
        print(f'X_{i}{j} = {value(model.X[i,j])}')

X_11 = 1.0
X_12 = 0.0
X_13 = 0.0
X_14 = 0.0
X_15 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 0.0
X_24 = 0.0
X_25 = 1.0
X_31 = 0.0
X_32 = 0.0
X_33 = 0.0
X_34 = 0.0
X_35 = 1.0


In [ ]:
# Import Pyomo
from pyomo.environ import *

# Data: inputted by human
I = list(range(1,25))     # Available items
B = list(range(1,16))  # Available boxes
W = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
C = 9               # The capacity of each box
M = 15               # Maximum number of boxes

# Create a concrete model
model = ConcreteModel()

# Decision variables
model.X = Var(((i,j) for i in I for j in B), within=Binary)

# Objective function
model.obj = Objective(expr=sum(model.X[i,j] for i in I for j in B), sense=minimize)

# Constraints
model.item_placement = ConstraintList()
for i in I:
    model.item_placement.add(sum(model.X[i,j] for j in B) == 1)

model.box_capacity = ConstraintList()
for j in B:
    model.box_capacity.add(sum(W[i]*model.X[i,j] for i in I) <= C)

model.box_limit = Constraint(expr=sum(model.X[i,j] for i in I for j in B) <= M )

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of decision variables


{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 40, 'Number of variables': 360, 'Number of nonzeros': 1080, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.007010221481323242}]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**